In [1]:
import gym
from gym import spaces
import numpy as np

class WarehouseOrientationEnv(gym.Env):
    """
    Magazyn NxN z orientacją robota:
      - Stan: [robot_x, robot_y, target_x, target_y, orientation_index]
      - Akcje: 0=obrót w lewo o alpha, 1=obrót w prawo o alpha, 2=ruch do przodu
    """
    def __init__(self, n=5, alpha=90, max_steps=100):
        super().__init__()  # Wywołanie init z gym.Env

        self.n = n
        self.alpha = alpha
        self.max_steps = max_steps
        
        # Ilu "kroków" orientacji (np. 360 / 90 = 4)
        assert 360 % alpha == 0, "Kąt alpha musi dzielić 360 (np. 90, 45, 60)."
        self.num_orientations = 360 // alpha

        # Przestrzeń akcji: 3 akcje
        self.action_space = spaces.Discrete(3)

        # Przestrzeń stanów (Box z 5 wymiarami):
        #   robot_x, robot_y, target_x, target_y, orientation_index
        low = np.array([0, 0, 0, 0, 0], dtype=np.float32)
        high = np.array([n-1, n-1, n-1, n-1, self.num_orientations - 1], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # Pola do przechowywania stanu
        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        self.orientation = 0  # indeks 0..(num_orientations-1)
        self.current_step = 0

        # Dodatkowe flagi
        self.terminated = False
        self.reward = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)  # możliwe wywołanie, jeśli Gym >=0.26
        self.current_step = 0
        self.terminated = False
        self.reward = 0

        # Losujemy pozycję robota
        self.robot_x = np.random.randint(self.n)
        self.robot_y = np.random.randint(self.n)

        # Losujemy pozycję celu (inną niż robota)
        while True:
            self.target_x = np.random.randint(self.n)
            self.target_y = np.random.randint(self.n)
            if (self.target_x != self.robot_x) or (self.target_y != self.robot_y):
                break

        # Losujemy orientację
        self.orientation = np.random.randint(self.num_orientations)

        return self._get_obs()

    def step(self, action):
        """
        Zwraca krotkę (observation, reward, terminated, truncated, info).
        W Gym <=0.25 może to być (observation, reward, done, info).
        """
        self.reward = 0
        old_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        # Wykonanie akcji
        if action == 0:
            # Obrót w lewo o alpha
            self.orientation = (self.orientation - 1) % self.num_orientations
        elif action == 1:
            # Obrót w prawo o alpha
            self.orientation = (self.orientation + 1) % self.num_orientations
        elif action == 2:
            # Ruch do przodu w kierunku aktualnej orientacji
            self._move_forward()
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Niewielka kara za każdy ruch (np. -0.0001)
        self.reward += -0.0001

        new_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        # Drobna nagroda za zbliżenie się do celu:
        if new_distance < old_distance:
            self.reward += 0.1
        elif new_distance > old_distance:
            # Kara za oddalenie się od celu
            self.reward -= 0.1

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        # Inkrementacja kroku
        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated, {}

    def render(self, mode='human'):
        """
        Prosta wizualizacja w konsoli.
        'R' = robot, 'T' = cel, '.' = puste pole
        """
        grid = [["." for _ in range(self.n)] for _ in range(self.n)]

        # Robot
        grid[self.robot_y][self.robot_x] = "R"
        # Cel
        grid[self.target_y][self.target_x] = "T"

        print("=" * (2*self.n))
        for row in grid:
            print(" ".join(row))
        print("=" * (2*self.n))
        orientation_deg = self.orientation * self.alpha
        print(f"Orientacja: index={self.orientation}, kąt={orientation_deg}°")

    def _get_obs(self):
        """
        Zwraca obserwację:
           [robot_x, robot_y, target_x, target_y, orientation_index]
        """
        return np.array([
            self.robot_x,
            self.robot_y,
            self.target_x,
            self.target_y,
            self.orientation
        ], dtype=np.float32)

    def _move_forward(self):
        """
        Ruch robota do przodu o 1 kratkę w zależności od orientation.
        Jeśli ruch poza planszę -> kara i zakończenie epizodu.
        """
        angle_deg = self.orientation * self.alpha
        angle_rad = np.deg2rad(angle_deg)

        dx = int(round(np.cos(angle_rad)))
        dy = int(round(np.sin(angle_rad)))

        new_x = self.robot_x + dx
        new_y = self.robot_y + dy

        # Sprawdzenie granic
        if 0 <= new_x < self.n and 0 <= new_y < self.n:
            self.robot_x = new_x
            self.robot_y = new_y
        else:
            # Kara za wyjście poza obszar i zakończenie
            self.reward -= 0.5
            self.terminated = True

In [2]:
import gym
from gym import spaces
import numpy as np

class WarehouseGridEnv2(gym.Env):
    """
    Magazyn NxN z klasycznym sterowaniem (4 akcje):
      - Stan: [robot_x, robot_y, target_x, target_y, dist_up, dist_down, dist_left, dist_right, dist_diag]
      - Akcje: 0=góra, 1=dół, 2=lewo, 3=prawo

    Dodatkowo:
      - `grid` opisuje siatkę (n x n), w której 0 to wolne pole, 1 – pole zajęte.
      - Wejście na pole o wartości 1 traktujemy jak kolizję.
    """
    def __init__(self, 
                 grid=None,
                 n=5, 
                 max_steps=100):
        super().__init__()

        self.max_steps = max_steps
        
        # Jeżeli przekazano siatkę, to z niej wyznacz rozmiar, w przeciwnym razie stwórz wolny obszar NxN
        if grid is not None:
            self.grid = grid
            self.n = self.grid.shape[0]
        else:
            self.n = n
            # Domyślnie tworzymy wolną siatkę, same zera
            self.grid = np.zeros((self.n, self.n), dtype=int)

        # Przestrzeń akcji: 4
        self.action_space = spaces.Discrete(4)

        # Observation space: 9 wymiarów (4 współrzędne + 5 sensorów)
        low = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32)
        high = np.array([
            self.n - 1,  # robot_x
            self.n - 1,  # robot_y
            self.n - 1,  # target_x
            self.n - 1,  # target_y
            self.n - 1,  # dist_up
            self.n - 1,  # dist_down
            self.n - 1,  # dist_left
            self.n - 1,  # dist_right
            self.n - 1,  # dist_diag
        ], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # Zmienna do śledzenia stanu (inicjalizowana w reset)
        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        self.current_step = 0
        self.terminated = False
        self.reward = 0

    def _distance_to_wall(self, start_x, start_y, dx, dy):
        """
        Zwraca liczbę wolnych pól w kierunku (dx, dy) do najbliższej ściany (przeszkody) 
        lub krawędzi siatki, startując od (start_x, start_y).
        """
        step_count = 0
        x, y = start_x, start_y
        while True:
            x_next = x + dx
            y_next = y + dy
            # Sprawdzamy krawędź
            if not (0 <= x_next < self.n and 0 <= y_next < self.n):
                return step_count
            # Sprawdzamy przeszkodę
            if self.grid[y_next, x_next] == 1:
                return step_count
            step_count += 1
            x, y = x_next, y_next

    def _get_sensors(self):
        rx, ry = self.robot_x, self.robot_y
        dist_up    = self._distance_to_wall(rx, ry,  0, -1)
        dist_down  = self._distance_to_wall(rx, ry,  0,  1)
        dist_left  = self._distance_to_wall(rx, ry, -1,  0)
        dist_right = self._distance_to_wall(rx, ry,  1,  0)
        dist_diag  = self._distance_to_wall(rx, ry, -1, -1)  # przykład: ukos w górę-lewo
        return np.array([dist_up, dist_down, dist_left, dist_right, dist_diag], dtype=np.float32)

    def _get_obs(self):
        # Podstawowe obserwacje (4 wartości)
        podstawowe = np.array([self.robot_x, self.robot_y, 
                               self.target_x, self.target_y], dtype=np.float32)
        # Sensory (5 wartości)
        sensory = self._get_sensors()
        # Łączymy w jeden wektor stanu
        return np.concatenate([podstawowe, sensory])

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.terminated = False
        self.reward = 0

        # Losowanie pozycji robota (tylko na wolnym polu)
        while True:
            rx = np.random.randint(self.n)
            ry = np.random.randint(self.n)
            if self.grid[ry, rx] == 0:  # wolne pole
                self.robot_x = rx
                self.robot_y = ry
                break

        # Losowanie pozycji celu (tylko na wolnym polu i różne od pozycji robota)
        while True:
            tx = np.random.randint(self.n)
            ty = np.random.randint(self.n)
            if (self.grid[ty, tx] == 0 and
                (tx != self.robot_x or ty != self.robot_y)):
                self.target_x = tx
                self.target_y = ty
                break

        return self._get_obs()

    def step(self, action):
        old_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        # Potencjalna nowa pozycja
        if action == 0:   # góra
            new_x = self.robot_x
            new_y = self.robot_y - 1
        elif action == 1: # dół
            new_x = self.robot_x
            new_y = self.robot_y + 1
        elif action == 2: # lewo
            new_x = self.robot_x - 1
            new_y = self.robot_y
        elif action == 3: # prawo
            new_x = self.robot_x + 1
            new_y = self.robot_y
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Sprawdzenie warunków kolizji lub wyjścia poza obszar
        if not (0 <= new_x < self.n and 0 <= new_y < self.n):
            # Kara za wyjście poza obszar
            self.reward = -0.5
            self.terminated = True
        elif self.grid[new_y, new_x] == 1:
            # Kara za kolizję z polem zajętym
            self.reward = -0.5
            self.terminated = True
        else:
            # Jeżeli pole jest wolne, aktualizujemy pozycję
            self.robot_x = new_x
            self.robot_y = new_y
            # Domyślna, niewielka kara za ruch
            self.reward = -0.0001

            new_distance = np.linalg.norm([self.robot_x - self.target_x,
                                           self.robot_y - self.target_y])
            # Nagroda/kara za przybliżenie/oddalenie się od celu
            if new_distance < old_distance:
                self.reward += 0.1
            elif new_distance > old_distance:
                self.reward -= 0.1

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated, {}

    def render(self, mode='human'):
        """
        Tekstowa wizualizacja środowiska:
        'R' = robot, 'T' = cel, 'X' = przeszkoda, '.' = puste pole.
        """
        grid_render = []
        for y in range(self.n):
            row_symbols = []
            for x in range(self.n):
                if self.grid[y, x] == 1:
                    symbol = "X"
                else:
                    symbol = "."
                
                if x == self.robot_x and y == self.robot_y:
                    symbol = "R"
                if x == self.target_x and y == self.target_y:
                    symbol = "T"
                row_symbols.append(symbol)
            grid_render.append(row_symbols)

        print("=" * (2 * self.n))
        for row in grid_render:
            print(" ".join(row))
        print("=" * (2 * self.n))


In [3]:
import gym
from gym import spaces
import numpy as np

class WarehouseGridEnv(gym.Env):
    """
    Magazyn NxN z klasycznym sterowaniem (4 akcje):
      - Stan: [robot_x, robot_y, target_x, target_y]
      - Akcje: 0=góra, 1=dół, 2=lewo, 3=prawo
    """
    def __init__(self, n=5, max_steps=100):
        super().__init__()  # wywołanie __init__ z gym.Env

        self.n = n
        self.max_steps = max_steps

        # Przestrzeń akcji: 4
        self.action_space = spaces.Discrete(4)

        # Stan: (robot_x, robot_y, target_x, target_y)
        low = np.array([0, 0, 0, 0], dtype=np.float32)
        high = np.array([n-1, n-1, n-1, n-1], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        self.current_step = 0
        self.terminated = False
        self.reward = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.terminated = False
        self.reward = 0

        # Losowanie pozycji robota
        self.robot_x = np.random.randint(self.n)
        self.robot_y = np.random.randint(self.n)

        # Losowanie pozycji celu
        while True:
            self.target_x = np.random.randint(self.n)
            self.target_y = np.random.randint(self.n)
            if (self.target_x != self.robot_x) or (self.target_y != self.robot_y):
                break

        return self._get_obs()

    def step(self, action):
        old_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        if action == 0:   # góra
            new_x = self.robot_x
            new_y = self.robot_y - 1
        elif action == 1: # dół
            new_x = self.robot_x
            new_y = self.robot_y + 1
        elif action == 2: # lewo
            new_x = self.robot_x - 1
            new_y = self.robot_y
        elif action == 3: # prawo
            new_x = self.robot_x + 1
            new_y = self.robot_y
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Sprawdzenie granic
        if not (0 <= new_x < self.n and 0 <= new_y < self.n):
            # Kara za wyjście poza obszar
            self.reward = -0.5
            self.terminated = True
        else:
            # Aktualizacja pozycji
            self.robot_x = new_x
            self.robot_y = new_y
            # Domyślna niewielka kara za ruch
            self.reward = -0.0001

            new_distance = np.linalg.norm([self.robot_x - self.target_x,
                                           self.robot_y - self.target_y])
            # Nagroda/kara za przybliżenie/oddalenie
            if new_distance < old_distance:
                self.reward += 0.1
            elif new_distance > old_distance:
                self.reward -= 0.1

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated, {}

    def render(self, mode='human'):
        """
        Tekstowa wizualizacja środowiska:
        'R' = robot, 'T' = cel, '.' = puste pole.
        """
        grid = [["." for _ in range(self.n)] for _ in range(self.n)]
        grid[self.robot_y][self.robot_x] = "R"
        grid[self.target_y][self.target_x] = "T"

        print("=" * (2 * self.n))
        for row in grid:
            print(" ".join(row))
        print("=" * (2 * self.n))

    def _get_obs(self):
        return np.array([self.robot_x, self.robot_y,
                         self.target_x, self.target_y], dtype=np.float32)


In [4]:
import gym
from gym import spaces
import numpy as np

class WarehouseGridEnv(gym.Env):
    """
    Magazyn NxN z klasycznym sterowaniem:
      - Stan: [r_x, r_y, t_x, t_y]
      - Akcje:
        0 = góra (y -= 1)
        1 = dół (y += 1)
        2 = lewo (x -= 1)
        3 = prawo (x += 1)
    """

    def __init__(self, n=5, max_steps=100):
        super(WarehouseGridEnv, self).__init__()
        
        self.n = n
        self.max_steps = max_steps

        # Przestrzeń akcji: 4 akcje
        #  0: góra
        #  1: dół
        #  2: lewo
        #  3: prawo
        self.action_space = spaces.Discrete(4)

        # Przestrzeń stanów (r_x, r_y, t_x, t_y) - wszystko w [0, n-1]
        low = np.array([0, 0, 0, 0], dtype=np.float32)
        high = np.array([n-1, n-1, n-1, n-1], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # Pola do przechowywania stanu
        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        
        self.current_step = 0
        self.reward = 0
        self.terminated = False

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.current_step = 0
        self.terminated = False
        
        # Losujemy pozycję robota
        self.robot_x = np.random.randint(self.n)
        self.robot_y = np.random.randint(self.n)
        self.reward = 0
        
        # Losujemy pozycję celu (inną niż robota)
        while True:
            self.target_x = np.random.randint(self.n)
            self.target_y = np.random.randint(self.n)
            if (self.target_x != self.robot_x) or (self.target_y != self.robot_y):
                break

        return self._get_obs()

    def step(self, action):
        self.reward = 0

        # Odległość przed akcją
        old_distance = np.linalg.norm([
            self.robot_x - self.target_x,
            self.robot_y - self.target_y
        ])

        # Wykonujemy ruch w zależności od akcji
        if action == 0:   # góra
            new_x = self.robot_x
            new_y = self.robot_y - 1
        elif action == 1: # dół
            new_x = self.robot_x
            new_y = self.robot_y + 1
        elif action == 2: # lewo
            new_x = self.robot_x - 1
            new_y = self.robot_y
        elif action == 3: # prawo
            new_x = self.robot_x + 1
            new_y = self.robot_y
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Sprawdzamy, czy wychodzimy poza magazyn
        if not (0 <= new_x < self.n and 0 <= new_y < self.n):
            # Kara za wyjście poza obszar + zakończenie epizodu
            self.reward = -0.5
            self.terminated = True
        else:
            # Aktualizujemy pozycję
            self.robot_x = new_x
            self.robot_y = new_y

        # Domyślna niewielka kara (koszt) za ruch
        self.reward += -0.0001

        # Nowa odległość
        new_distance = np.linalg.norm([
            self.robot_x - self.target_x,
            self.robot_y - self.target_y
        ])

        # Nagroda/kara w zależności od tego, czy zbliżyliśmy się do celu
        if not self.terminated:  # Tylko jeśli jeszcze nie zakończono
            if new_distance < old_distance:
                self.reward += 0.1   # przybliżenie
            elif new_distance > old_distance:
                self.reward -= 0.1   # oddalenie

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        # Inkrementujemy krok i sprawdzamy limit
        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated

    def render(self, mode='human'):
        """
        Prosta wizualizacja tekstowa w konsoli.
        'R' = robot
        'T' = cel
        '.' = puste
        """
        grid = [["." for _ in range(self.n)] for _ in range(self.n)]

        # Pozycja robota
        grid[self.robot_y][self.robot_x] = "R"
        # Pozycja celu
        grid[self.target_y][self.target_x] = "T"

        print("="*(2*self.n))
        for row in grid:
            print(" ".join(row))
        print("="*(2*self.n))

    def _get_obs(self):
        """
        Zwracamy obserwację (r_x, r_y, t_x, t_y).
        """
        return np.array([
            self.robot_x,
            self.robot_y,
            self.target_x,
            self.target_y
        ], dtype=np.float32)


In [ ]:
import numpy as np
from tensorflow.keras.losses import MeanSquaredError
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from collections import deque
import random
import pickle
import math
import plotly.graph_objects as go
from IPython.display import display, clear_output

# =============================
# 1. Definicje Stałych i Hiperparametrów
# =============================

GAMMA = 0.9
EPSILON_START = 0.2
EPSILON_MIN = 0.0
EPSILON_DECAY = 0.9  # Zmniejszanie epsilon stopniowo
BATCH_SIZE = 24 # Typowy rozmiar batcha dla DQN
LEARNING_RATE = 0.0001
MEMORY_SIZE = 200000    # Rozmiar pamięci replay
EPISODES = 200000

# =============================
# 2. Funkcja Wczytująca Mapę
# =============================

def wczytaj_cyfry_z_pliku_jako_macierz(nazwa_pliku):
    macierz = []
    with open(nazwa_pliku, 'r') as plik:
        for linia in plik:
            linia = linia.strip()
            if not linia:
                continue
            wiersz = [int(znak) for znak in linia]
            macierz.append(wiersz)
    return macierz

# Przykładowa mapa (ręcznie):
my_grid = np.array([
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
])
print("Mapa załadowana:\n", my_grid)

# =============================
# 3. Wybór Środowiska
# =============================

ENV_TYPE = "grid"  # "orientation" lub "grid"

if ENV_TYPE == "orientation":
    env = WarehouseOrientationEnv(n=64, alpha=90, max_steps=200)
else:
    env = WarehouseGridEnv2(n=8, max_steps=32, grid=my_grid)

# =============================
# 4. Standaryzacja Stanów
# =============================

MEANS = {
    'x': 4.5,
    'y':  4.5,
    'tx':  4.5,
    'ty':  4.5,
}
STDS = {
    'x': 5.25,
    'y': 5.25,
    'tx': 5.25,
    'ty': 5.25,
}

def transform_state_orientation(obs):
    """
    Zakładamy obs ma postać (5,):
      [x, y, tx, ty, orientation]   gdzie orientation ∈ {0,1,2,3}
    Zwracamy [x_norm, y_norm, tx_norm, ty_norm, sin(theta), cos(theta)]
    """
    obs = np.array(obs, dtype=float)
    # Standaryzacja współrzędnych
    x = (obs[0] - MEANS['x']) / STDS['x']
    y = (obs[1] - MEANS['y']) / STDS['y']
    tx = (obs[2] - MEANS['tx']) / STDS['tx']
    ty = (obs[3] - MEANS['ty']) / STDS['ty']
    orientation = obs[4]
    angle_deg = orientation * 90.0
    angle_rad = np.deg2rad(angle_deg)
    sin_v = np.sin(angle_rad)
    cos_v = np.cos(angle_rad)
    new_obs = np.array([x, y, tx, ty, sin_v, cos_v], dtype=float)
    return new_obs

def transform_state_grid(obs):
    """
    Zakładamy obs ma postać (4,) lub więcej, np. [x, y, tx, ty, ...].
    Tu dla przykładu weźmy tylko x, y, tx, ty.
    """
    obs = np.array(obs, dtype=float)
    x = (obs[0] - MEANS['x']) / STDS['x']
    y = (obs[1] - MEANS['y']) / STDS['y']
    tx = (obs[2] - MEANS['tx']) / STDS['tx']
    ty = (obs[3] - MEANS['ty']) / STDS['ty']
    new_obs = np.array([x, y, tx, ty], dtype=float)
    return new_obs

if ENV_TYPE == "orientation":
    transform_state_fn = transform_state_orientation
    state_size = 6
else:
    transform_state_fn = transform_state_grid
    state_size = 4

# =============================
# 5. Definicje Modelu Q-network i Target Network
# =============================

def build_q_network(state_size, action_size):
    model = models.Sequential()
    model.add(layers.Dense(128, input_shape=(state_size,), activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128,  activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    return model

def build_target_network(model):
    target = models.clone_model(model)
    target.set_weights(model.get_weights())
    return target

# =============================
# 6. Inicjalizacja Pamięci Replay
# =============================

memory = deque(maxlen=MEMORY_SIZE)

# =============================
# 7. Inicjalizacja Epsilon i Monitoringu
# =============================

epsilon = EPSILON_START
all_rewards = []
loss_values = []

# Zmienna do wyliczania uśrednionego Loss
smoothed_loss_values =[0] 

# Zmienna do wyliczania normy wag
weight_values = []

# Zmienna do wyliczania uśrednionej nagrody
smoothed_reward_values = []

# =============================
# 8. Ustalenie action_size
# =============================

action_size = env.action_space.n

# =============================
# 9. Inicjalizacja Sieci Q i Sieci Target
# =============================

q_network = build_q_network(state_size, action_size)
target_network = build_target_network(q_network)

# Kompilacja sieci (aby można było np. wywoływać model(...) w trybie treningowym)
optimizer = optimizers.Adam(learning_rate=LEARNING_RATE,)
loss_function = MeanSquaredError()

# =============================
# 10. Przygotowanie osobnych wykresów
# =============================

# 1) Wykres Loss
fig_loss = go.FigureWidget()
fig_loss.add_scatter(y=smoothed_loss_values, mode='lines', name='Loss (MA)')
fig_loss.update_layout(
    title='Training: Loss (Moving Average)',
    xaxis_title='Training Steps',
    yaxis_title='Loss',
    template='plotly_white'
)

# 2) Wykres Weight (norma wag)
fig_weight = go.FigureWidget()
fig_weight.add_scatter(y=weight_values, mode='lines', name='WeightNorm')
fig_weight.update_layout(
    title='Training: Weight Norm',
    xaxis_title='Training Steps',
    yaxis_title='WeightNorm',
    template='plotly_white'
)

# 3) Wykres Reward (uśredniony)
fig_reward = go.FigureWidget()
fig_reward.add_scatter(y=smoothed_reward_values, mode='lines', name='Reward (MA)')
fig_reward.update_layout(
    title='Training: Reward (Moving Average)',
    xaxis_title='Episode',
    yaxis_title='Reward',
    template='plotly_white'
)

display(fig_loss, fig_weight, fig_reward)

# =============================
# 11. Główna Pętla Treningowa
# =============================

training_steps = 0
window_size = 50  # wielkość okna do średniej kroczącej

for episode in range(1, EPISODES + 1):
    obs = env.reset()                      
    obs = transform_state_fn(obs)          
    terminated = False
    truncated = False
    total_reward = 0

    while not (terminated or truncated):
        # Epsilon-greedy
        if np.random.rand() < epsilon or len(memory)<4000:
            action = np.random.randint(action_size)
        else:
            q_values = q_network.predict(obs[np.newaxis, :], verbose=0)
            action = np.argmax(q_values[0])
        # Wykonanie akcji w środowisku
        next_obs, reward, terminated, truncated, info = env.step(action)
        next_obs = transform_state_fn(next_obs)
        total_reward += reward

        # Zapis do pamięci
        memory.append((obs, action, reward, next_obs, terminated))
        obs = next_obs

    # Uczenie, jeśli mamy dostatecznie dużą pamięć
    if len(memory) >= 4000:
        # 1. Pobieramy batch z pamięci (losowe próbki)
        minibatch = random.sample(memory, BATCH_SIZE)

        # 2. Konwertujemy próbki do macierzy
        states_mb      = np.array([m[0] for m in minibatch])
        actions_mb     = np.array([m[1] for m in minibatch])
        rewards_mb     = np.array([m[2] for m in minibatch])
        next_states_mb = np.array([m[3] for m in minibatch])
        dones_mb       = np.array([m[4] for m in minibatch])

        # 3. Przewidywania sieci target i sieci Q
        q_next = target_network.predict(next_states_mb, verbose=0)
        q_targets = q_network.predict(states_mb, verbose=0)

        # 4. Ustalenie docelowych wartości Q (q_targets)
        for i in range(BATCH_SIZE):
            if dones_mb[i]:
                q_targets[i, actions_mb[i]] = rewards_mb[i]
            else:
                q_targets[i, actions_mb[i]] = rewards_mb[i] + GAMMA * np.max(q_next[i])

        # 5. Pojedynczy krok uczenia (Gradient Descent)
        with tf.GradientTape() as tape:
            predictions = q_network(states_mb, training=True)
            loss_value = loss_function(q_targets, predictions)
            loss_values.append(loss_value.numpy())
        gradients = tape.gradient(loss_value, q_network.trainable_variables)
        optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))
        w_norm = 0.0
        for var in q_network.trainable_variables:
            w_norm += np.sum(var.numpy()**2)
            w_norm = np.sqrt(w_norm)
        weight_values.append(w_norm)
        # 6. Logowanie
       
            # ==============================
            # Trenowanie sieci "ręcznie" z użyciem GradientTape
            # ==============================
            # with tf.GradientTape() as tape:
            #     predictions = q_network(states_mb, training=True)
            #     loss = loss_function(q_targets, predictions)

            # gradients = tape.gradient(loss, q_network.trainable_variables)
            # optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))

            # # --- Zapis i obliczanie średniej kroczącej dla Loss ---
            # loss_value = loss.numpy()
            # loss_values.append(loss_value)
        
    if len(loss_values) < window_size:
        ma_loss = np.mean(loss_values)
    else:
        ma_loss = np.mean(loss_values[-window_size:])
    smoothed_loss_values.append(ma_loss)

    # --- Obliczanie normy wag (np. L2 wszystkich wag) ---
   

    # --- Aktualizacja dwóch wykresów w czasie rzeczywistym ---
    with fig_loss.batch_update():
        fig_loss.data[0].y = smoothed_loss_values
        fig_loss.update_layout(xaxis=dict(range=[0, len(smoothed_loss_values)]))

    with fig_weight.batch_update():
        fig_weight.data[0].y = weight_values
        fig_weight.update_layout(xaxis=dict(range=[0, len(weight_values)]))

    # Zmniejszamy epsilon
    if epsilon > EPSILON_MIN:
        epsilon *= EPSILON_DECAY
        epsilon = max(epsilon, EPSILON_MIN)

    # Zapisujemy nagrodę za epizod
    all_rewards.append(total_reward)

    # Okresowe kopiowanie wag do sieci target
    #tau = 0.02  # przykładowa wartość
    target_weights = target_network.get_weights()
    source_weights = q_network.get_weights()
    new_weights = []

    # Mieszamy wagi: theta_target := tau * theta_source + (1 - tau) * theta_target
    if episode % 50 == 0:
       target_network.set_weights(q_network.get_weights())

    # Co pewną liczbę epizodów wypisz informacje i aktualizuj wykres nagrody
    if episode % 2 == 0:
        avg_reward = np.mean(all_rewards[-1:])
        print(f"Epizod: {episode}/{EPISODES}, "
              f"AVG ostatnich 50 ep.: {avg_reward:.3f}, "
              f"epsilon: {epsilon:.3f}, ")

    # --- Aktualizacja wykresu nagrody po KAŻDYM zakończonym epizodzie ---
    if len(all_rewards) < window_size:
        ma_reward = np.mean(all_rewards)
    else:
        ma_reward = np.mean(all_rewards[-window_size:])
    smoothed_reward_values.append(ma_reward)

    with fig_reward.batch_update():
        fig_reward.data[0].y = smoothed_reward_values
        fig_reward.update_layout(xaxis=dict(range=[0, len(smoothed_reward_values)]))

print("Trening zakończony!")


Mapa załadowana:
 [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 1 1 1 1 0 0]
 [0 0 1 1 1 1 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


C:\Users\Filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



FigureWidget({
    'data': [{'mode': 'lines', 'name': 'Loss', 'type': 'scatter', 'uid': '6c4d2dcd-9f32-4909-88a2-d24a9976ecfb', 'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training: Loss'},
               'xaxis': {'title': {'text': 'Training Steps'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'GradNorm',
              'type': 'scatter',
              'uid': '8d0f5795-f285-47d9-9ea2-81b261d50355',
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training: Gradient Norm'},
               'xaxis': {'title': {'text': 'Training Steps'}},
               'yaxis': {'title': {'text': 'GradNorm'}}}
})

NameError: name 'memory' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Wczytujemy nasz wytrenowany model
new_model = tf.keras.models.load_model('trained_dqn_model.h5')
trained_model = new_model

def test_agent(env, model, n_episodes=5):
    """
    Wykonujemy n_episodes epizodów i zapisujemy trace w postaci:
    (obs, action, reward, next_obs) dla każdego kroku.
    """
    all_episode_rewards = []
    all_traces = []  # aby przechować szczegółowe info do ewentualnej wizualizacji

    for episode_i in range(n_episodes):
        obs_5d = env.reset()
        obs_6d = transform_state(obs_5d)  # Funkcja jak w Twoim kodzie

        episode_reward = 0
        episode_trace = []

        terminated = False
        truncated = False

        while not (terminated or truncated):
            # Wybór akcji - tutaj greedy, bo chcemy pokazać, co robi nauczony agent
            q_values = model.predict(obs_6d[np.newaxis, :], verbose=0)
            action = np.argmax(q_values[0])

            # Krok w środowisku
            next_obs_5d, reward, terminated, truncated = env.step(action)
            next_obs_6d = transform_state(next_obs_5d)

            episode_reward += reward

            # Zapis do trace
            episode_trace.append({
                'obs': obs_5d,
                'action': action,
                'reward': reward,
                'next_obs': next_obs_5d
            })

            # Przechodzimy do następnego stanu
            obs_5d = next_obs_5d
            obs_6d = next_obs_6d

        all_episode_rewards.append(episode_reward)
        all_traces.append(episode_trace)

    # Podsumowanie wyników
    avg_reward = np.mean(all_episode_rewards)
    print(f"Test: średni reward z {n_episodes} epizodów = {avg_reward:.2f}")

    return all_episode_rewards, all_traces

# Uruchamiamy testy
env = WarehouseOrientationEnv(n=64, max_steps=200)
test_rewards, traces = test_agent(env, trained_model, n_episodes=5)

In [ ]:
import matplotlib.pyplot as plt

def visualize_episode(episode_trace):
    """
    Wizualizuje pozycje agenta i celu w kolejnych krokach epizodu.
    Załóżmy, że `episode_trace` to lista słowników, gdzie:
        step['obs'] = [agent_x, agent_y, target_x, target_y, ...]
    """

    # Listy do przechowania kolejnych pozycji agenta i celu
    agent_positions = []
    target_positions = []
    a=print(episode_trace)
    for step in episode_trace:
        # Wyciągamy z obs: [agent_x, agent_y, target_x, target_y, ...]
      agent_x, agent_y   = step['obs'][0], step['obs'][1]
      target_x, target_y = step['obs'][2], step['obs'][3]

      agent_positions.append((agent_x, agent_y))
      target_positions.append((target_x, target_y))

    # Rozbijamy na osobne listy (x, y) — jeśli target się porusza, to zobaczymy jego ścieżkę
    agent_xs  = [pos[0] for pos in agent_positions]
    agent_ys  = [pos[1] for pos in agent_positions]
    target_xs = [pos[0] for pos in target_positions]
    target_ys = [pos[1] for pos in target_positions]

    # Rysujemy ścieżkę agenta
    plt.plot(agent_xs, agent_ys, 'bo-', label='Agent')

    # Rysujemy ścieżkę (lub pozycje) celu
    # Jeśli cel jest statyczny, zobaczysz same nakładające się punkty.
    plt.plot(target_xs, target_ys, 'rx--', label='Cel')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Ścieżka agenta i pozycja celu')
    plt.legend()
    plt.grid(True)
    plt.show()

# Załóżmy, że mamy listę epizodów `traces`,
# gdzie np. traces[0] to trace z pierwszego epizodu
episode_trace = traces[0]  # bierzemy pierwszy epizod z listy

# Wywołujemy wizualizację
visualize_episode(episode_trace)